In [ ]:
!pip install gradio --upgrade


In [ ]:
# install Google Text to Speech library
!pip install gtts

In [ ]:
!pip install load_dotenv

In [ ]:
import gradio as gr
from transformers import pipeline
from dotenv import load_dotenv
import os
import subprocess
import torch
#Google Text to Speech
from gtts import gTTS
import tempfile

In [ ]:
# print(torch.__version__)

In [ ]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY = # to fill in here

In [ ]:
# convert audio file to a different format using ffmpeg

# def convert_audio(input_filepath, output_filepath):
#     #run ffmpeg and convert the audio file
#     command = ['ffmpeg', '-i', input_filepath, output_filepath]

#     #run the command
#     subprocess.run(command, check=True)

In [ ]:
#input and output file paths and call the function

#skip this step if you have already converted
# input_file = 'Resources\Recording.m4a'
# output_file = 'Resources\Recording.wav'
# convert_audio(input_file, output_file)

In [ ]:
# This cell from class exercise for reference
# Initialize the pipeline to translate using the t5-base model.
# translator = pipeline("translation", model="t5-base")

In [ ]:
# This function transcribes audio to text using Whisper
def transcribe_audio(audio_filepath):
    try:
        transcription_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-large")
        transcription_result = transcription_pipeline(audio_filepath)
        transcribed_text = transcription_result['text']
        return transcribed_text
    except Exception as e:
        print(f"an error occured: {e}")
        return "Error in transcription"

In [ ]:
# audio_filepath = 'Resources\Recording.wav'
#transcribed_text = transcribe_audio(audio_file_path)

In [ ]:
# Define function to translate text to speech for output
def text_to_speech(text):
    tts = gTTS(text, lang='en')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
    tts.save(temp_file.name)
    return temp_file.name

In [ ]:
# create combined function with both transcribe audio and text_to_speech

def combined_function (audio_filepath):
    transcribed_text = transcribe_audio(audio_filepath)
    speech_file_path = text_to_speech(transcribed_text)
    return transcribed_text, speech_file_path


In [ ]:
# Create Gradio app to:
# 1. transcribe spoken audio to text
# 2. output transcribed text as speech

input_audio = gr.Audio(
            label="click on microphone to record audio",
            type="filepath",
            #vaveform options customize the color of the wave seen when recording/playing.
            waveform_options = gr.WaveformOptions(
                waveform_color="#01C6FF",
                waveform_progress_color="#0066B4",
                skip_length=2,
                show_controls=False,
            ),
)

app = gr.Interface(
    fn=combined_function,
    inputs=input_audio,
    outputs=[
        gr.Textbox(label="Transcribed audio"),
        gr.Audio(label="Text to speech output")],
    title="Audio Transcription and Text to Speech",
    description="click on the microphone to record audio, then receive transcription in text and speech.")

app.launch(show_error=True) #share=True) #uncomment share=True in google colab

In [ ]:
# Create Gradio app to transcribe spoken audio to text

app = gr.Interface(
    fn=transcribe_audio,
    inputs=[
        gr.Audio(label="upload audio file or click on microphone to record audio", type="filepath")],
    outputs=gr.Textbox(label="transcribed audio"))

app.launch(show_error=True)